## Tool for selecting valid segments

In [131]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, Output, VBox
import json
import matplotlib.dates as mdates

# Ensure that matplotlib plots inline
%matplotlib inline

## Loading and Segmentation

In [132]:
def load_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    # Create a list of DataFrames, one for each gesture
    dfs = []
    for item in data:
        df = pd.DataFrame({
            'timestamps': item['timestamps'],
            'xTimeSeries': item['xTimeSeries'],
            'yTimeSeries': item['yTimeSeries'],
            'zTimeSeries': item['zTimeSeries'],
        })
        df['timestamps'] = pd.to_datetime(df['timestamps'], unit='ms')  # Convert Unix ms to datetime
        dfs.append(df)
    return dfs

In [133]:
def segment_signal(df, window_size=100, overlap=50):
    segments = []
    start = 0
    end = start + window_size
    while end <= len(df):
        segments.append(df.iloc[start:end])
        start += (window_size - overlap)
        end = start + window_size
    return segments


## Visualization and labelling

In [134]:

def plot_and_label(dataframes, window_size=100, overlap=50):
    out = Output()
    segments = []
    gesture_boundaries = []  # To track the start of each gesture's segments
    current_boundary = 0

    for idx, df in enumerate(dataframes):
        gesture_segments = segment_signal(df, window_size, overlap)
        segments.extend([(segment.copy(), df, idx) for segment in gesture_segments])
        gesture_boundaries.append((current_boundary, current_boundary + len(gesture_segments)))
        current_boundary += len(gesture_segments)

    labels = ['Unlabeled'] * len(segments)  # Initialize labels to 'Unlabeled'
    current_segment_index = 0

    def plot_segment(index):
        segment, full_df, df_index = segments[index]
        current_gesture_index = df_index + 1
        total_gestures = len(dataframes)

        with out:
            out.clear_output(wait=True)
            plt.figure(figsize=(12, 6))
            plt.plot(full_df['timestamps'], full_df['xTimeSeries'], label='xTimeSeries', color='lightgray')
            plt.plot(full_df['timestamps'], full_df['yTimeSeries'], label='yTimeSeries', color='lightgray')
            plt.plot(full_df['timestamps'], full_df['zTimeSeries'], label='zTimeSeries', color='lightgray')
            plt.title(f'Full Time Series - Gesture {current_gesture_index} of {total_gestures}')
            plt.xlabel('Time')
            plt.ylabel('Acceleration')
            plt.legend()
            plt.grid(True)
            plt.axvspan(segment['timestamps'].iloc[0], segment['timestamps'].iloc[-1], color='yellow', alpha=0.3)
            plt.gca().xaxis.set_major_locator(mdates.SecondLocator())
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
            print(f"Labeling {index + 1}/{len(segments)}: Current label is '{labels[index]}'")

    def on_button_clicked(label):
        def callback(b):
            nonlocal current_segment_index
            if label == 'Next Gesture':
                # Skip to the next gesture's segments
                current_segment_index = next((end for start, end in gesture_boundaries if start <= current_segment_index < end), len(segments))
            else:
                labels[current_segment_index] = label if label != 'Discard' else 'Discarded'
                print(f"Set label '{labels[current_segment_index]}' for segment {current_segment_index + 1}")  # Immediate feedback

            current_segment_index += 1
            if current_segment_index < len(segments):
                plot_segment(current_segment_index)
            else:
                save_labeled_data()
        return callback

    def save_labeled_data():
        def convert_df_to_serializable_dict(segment, label):
            # Convert DataFrame to a dict and handle Timestamp serialization
            return {
                'timestamps': segment['timestamps'].apply(lambda x: x.isoformat() if pd.notnull(x) else None).tolist(),
                'xTimeSeries': segment['xTimeSeries'].tolist(),
                'yTimeSeries': segment['yTimeSeries'].tolist(),
                'zTimeSeries': segment['zTimeSeries'].tolist(),
                'label': label  # Attach the label
            }

        serialized_data = []
        for i, (segment, _, _) in enumerate(segments):
            if labels[i] != 'Discarded' and labels[i] != 'Unlabeled':
                data_dict = convert_df_to_serializable_dict(segment, labels[i])
                serialized_data.append(data_dict)
        
        with open('labeled_data.json', 'w') as file:
            json.dump(serialized_data, file)
        with out:
            out.clear_output()
            print(f"All segments labeled and saved, {len(serialized_data)} segments included.")

    button_labels = ['Circle', 'Clap', 'Other', 'Discard', 'Next Gesture']
    buttons = [Button(description=label) for label in button_labels]
    for button in buttons:
        button.on_click(on_button_clicked(button.description))

    display(VBox([HBox(buttons), out]))
    plot_segment(current_segment_index)

# Ensure segment_signal is defined correctly and plot_and_label is called appropriately.


In [135]:
filepath = 'circle_data.json'  # Update with your actual file path
dataframes = load_data(filepath)
plot_and_label(dataframes, 40,30)


Set label 'Other' for segment 1


Set label 'Discarded' for segment 2


Set label 'Discarded' for segment 3


Set label 'Discarded' for segment 4


Set label 'Clap' for segment 5
